In [5]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.utils import normalize

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
#from keras.layers.normalization import BatchNormalization # not implemented
from keras.callbacks import ReduceLROnPlateau, EarlyStopping#, ModelCheckpoint

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def accuracy_f_score(y_pred,y_true):
    print(f"Accuracy score: {round(accuracy_score(y_true, y_pred) * 100,2)}%")
    print('\033[92m' + f"F1 score: {f1_score(y_true, y_pred)}" + '\033[0m')
    

def in_city(y_pred):
    targets = []
    for pred in y_pred:
        if (3750901.5068 <= pred[0] <= 3770901.5069) and (-19268905.6133 <= pred[1] <= -19208905.6133):
            targets.append(1)
        else:
            targets.append(0)
    return targets

In [6]:
df1 = pd.read_csv('data_train/data_train.csv')
train_df = df1[df1["time_exit"][:].str[:2] == '15']

def journey_time(x,y):
    """
    Compute journey time in seconds.
    """
    x = pd.to_datetime(x)
    y = pd.to_datetime(y)
    return (y-x).total_seconds()

train_df["j_time"] = list(map(journey_time, train_df["time_entry"], train_df["time_exit"])) 

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [7]:
#train_df.set_index("trajectory_id", inplace=True)
train_df.set_index("trajectory_id", inplace=True)
X = train_df.drop(["x_exit","y_exit","vmax","vmin","vmean","time_entry","time_exit","hash","Unnamed: 0"], axis=1).values
y = [train_df["x_exit"].values, train_df["y_exit"].values]
y = np.transpose(np.array(y))

In [9]:
print(X[0])
print(y[0])
train_df.head()

[ 3.74494463e+06 -1.92818338e+07  9.62000000e+02]
[  3744785.44389591 -19281478.34777075]


,Unnamed: 0,hash,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,j_time
trajectory_id,,,,,,,,,,,,
traj_0000a8602cf2def930488dee7cdad104_1_5,5,0000a8602cf2def930488dee7cdad104_1,15:02:31,15:18:33,NaN,NaN,NaN,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,962.0
traj_0000cf177130469eeac79f67b6bcf3df_9_3,9,0000cf177130469eeac79f67b6bcf3df_9,15:00:32,15:29:48,1.149404,1.149404,1.149404,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,1756.0
traj_0001f97b99a80f18f62e2d44e54ef33d_3_1,11,0001f97b99a80f18f62e2d44e54ef33d_3,14:34:35,15:19:51,30.167742,30.167742,30.167742,3.758738e+06,-1.937594e+07,3.769687e+06,-1.914258e+07,2716.0
traj_0002124248b0ca510dea42824723ccac_31_10,20,0002124248b0ca510dea42824723ccac_31,15:28:54,15:28:54,NaN,NaN,NaN,3.767866e+06,-1.917797e+07,3.767866e+06,-1.917797e+07,0.0
traj_000219c2a6380c307e8bffd85b5e404b_23_16,28,000219c2a6380c307e8bffd85b5e404b_23,15:08:05,15:08:05,NaN,NaN,NaN,3.747641e+06,-1.922695e+07,3.747641e+06,-1.922695e+07,0.0


In [12]:
# from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X)
X = X.reshape((134037,X.shape[1],1))
# X.shape

1.12.0


In [172]:
#####Define the LSTM model#####

model = None
model = Sequential()

model.add(LSTM(16,activation='relu',return_sequences=False)) #False before Dense layer
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(Dense(512, activation='relu',kernel_initializer='normal'))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(Dense(2, activation='linear'))

optimiser = tf.keras.optimizers.Adam(lr=0.00001, decay=0) #lr=1e-2 = 31.82% valacc

EPOCHS = 100
BATCH_SIZE = 32
tf.set_random_seed(46)

In [174]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

checkpoint_name = 'logs/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1,
                             save_best_only = True, mode ='auto', save_weights_only=True)
model.compile(loss='mean_absolute_percentage_error', optimizer=optimiser, metrics=['accuracy']) # try different loss functions
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=410,shuffle=True)

"""
Smaller sample of data for testing to speed things up.
--Reset model checkpoint each time.
"""
model.fit(x_train[:20000],y_train[:20000], epochs=EPOCHS, validation_split=0.2, callbacks=[checkpoint])

Train on 16000 samples, validate on 4000 samples
Epoch 1/100
15968/16000 [============================>.] - ETA: 0s - loss: 69.7563 - acc: 1.0000
Epoch 00001: val_loss improved from inf to 40.93891, saving model to logs/Weights-001--40.93891.hdf5

Consider using a TensorFlow optimizer from `tf.train`.
16000/16000 [==============================] - 17s 1ms/step - loss: 69.6988 - acc: 1.0000 - val_loss: 40.9389 - val_acc: 1.0000
Epoch 2/100
15840/16000 [============================>.] - ETA: 0s - loss: 32.7647 - acc: 1.0000
Epoch 00002: val_loss improved from 40.93891 to 19.40261, saving model to logs/Weights-002--19.40261.hdf5

Consider using a TensorFlow optimizer from `tf.train`.
16000/16000 [==============================] - 4s 260us/step - loss: 32.6336 - acc: 1.0000 - val_loss: 19.4026 - val_acc: 1.0000
Epoch 3/100
15936/16000 [============================>.] - ETA: 0s - loss: 3.6971 - acc: 1.0000
Epoch 00003: val_loss improved from 19.40261 to 0.24797, saving model to logs/Weights

Epoch 18/100
15808/16000 [============================>.] - ETA: 0s - loss: 0.2436 - acc: 1.0000
Epoch 00018: val_loss did not improve from 0.23996
16000/16000 [==============================] - 4s 258us/step - loss: 0.2439 - acc: 1.0000 - val_loss: 0.2413 - val_acc: 1.0000
Epoch 19/100
15936/16000 [============================>.] - ETA: 0s - loss: 0.2445 - acc: 1.0000
Epoch 00019: val_loss did not improve from 0.23996
16000/16000 [==============================] - 4s 255us/step - loss: 0.2444 - acc: 1.0000 - val_loss: 0.2473 - val_acc: 1.0000
Epoch 20/100
15872/16000 [============================>.] - ETA: 0s - loss: 0.2427 - acc: 1.0000
Epoch 00020: val_loss did not improve from 0.23996
16000/16000 [==============================] - 4s 253us/step - loss: 0.2428 - acc: 1.0000 - val_loss: 0.2630 - val_acc: 1.0000
Epoch 21/100
15840/16000 [============================>.] - ETA: 0s - loss: 0.2432 - acc: 1.0000
Epoch 00021: val_loss did not improve from 0.23996
16000/16000 [==============

15904/16000 [============================>.] - ETA: 0s - loss: 0.2401 - acc: 1.0000
Epoch 00038: val_loss improved from 0.23775 to 0.23743, saving model to logs/Weights-038--0.23743.hdf5

Consider using a TensorFlow optimizer from `tf.train`.
16000/16000 [==============================] - 4s 263us/step - loss: 0.2401 - acc: 1.0000 - val_loss: 0.2374 - val_acc: 1.0000
Epoch 39/100
15872/16000 [============================>.] - ETA: 0s - loss: 0.2388 - acc: 1.0000
Epoch 00039: val_loss did not improve from 0.23743
16000/16000 [==============================] - 4s 256us/step - loss: 0.2389 - acc: 1.0000 - val_loss: 0.2413 - val_acc: 1.0000
Epoch 40/100
15904/16000 [============================>.] - ETA: 0s - loss: 0.2401 - acc: 1.0000
Epoch 00040: val_loss did not improve from 0.23743
16000/16000 [==============================] - 4s 254us/step - loss: 0.2401 - acc: 1.0000 - val_loss: 0.2393 - val_acc: 1.0000
Epoch 41/100
15808/16000 [============================>.] - ETA: 0s - loss: 0.24

Epoch 80/100
15968/16000 [============================>.] - ETA: 0s - loss: 0.2320 - acc: 1.0000
Epoch 00080: val_loss improved from 0.23110 to 0.23082, saving model to logs/Weights-080--0.23082.hdf5

Consider using a TensorFlow optimizer from `tf.train`.
16000/16000 [==============================] - 4s 257us/step - loss: 0.2319 - acc: 1.0000 - val_loss: 0.2308 - val_acc: 1.0000
Epoch 81/100
15968/16000 [============================>.] - ETA: 0s - loss: 0.2320 - acc: 1.0000
Epoch 00081: val_loss did not improve from 0.23082
16000/16000 [==============================] - 4s 251us/step - loss: 0.2320 - acc: 1.0000 - val_loss: 0.2336 - val_acc: 1.0000
Epoch 82/100
15872/16000 [============================>.] - ETA: 0s - loss: 0.2320 - acc: 1.0000
Epoch 00082: val_loss did not improve from 0.23082
16000/16000 [==============================] - 4s 249us/step - loss: 0.2321 - acc: 1.0000 - val_loss: 0.2309 - val_acc: 1.0000
Epoch 83/100
15968/16000 [============================>.] - ETA: 0s

In [175]:
weights_file = 'logs/Weights-095--0.22791.hdf5' # choose the best checkpoint 
model.load_weights(weights_file) # load it
model.compile(loss='mean_absolute_percentage_error', optimizer=optimiser, metrics=['accuracy'])

In [176]:
pred = model.predict(x_test)
pred_b = in_city(pred)
y_test_b = in_city(y_test)
accuracy_f_score(pred_b,y_test_b)

Accuracy score: 75.12%
F1 score: 0.6713961966696227


In [94]:
print(pred_b)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [4]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.utils import normalize

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
#from keras.layers.normalization import BatchNormalization # not implemented
from keras.callbacks import ReduceLROnPlateau, EarlyStopping#, ModelCheckpoint

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler

def journey_time(x,y):
    """
    Compute journey time in seconds.
    """
    x = pd.to_datetime(x)
    y = pd.to_datetime(y)
    return (y-x).total_seconds()

def accuracy_f_score(y_pred,y_true):
    print(f"Accuracy score: {round(accuracy_score(y_true, y_pred) * 100,2)}%")
    print('\033[92m' + f"F1 score: {f1_score(y_true, y_pred)}" + '\033[0m')
    

def in_city(y_pred):
    targets = []
    for pred in y_pred:
        if (3750901.5068 <= pred[0] <= 3770901.5069) and (-19268905.6133 <= pred[1] <= -19208905.6133):
            targets.append(1)
        else:
            targets.append(0)
    return targets

def sigmoid(x):
	e = np.exp(1)
	y = 1/(1+e**(-x))
	return y 


df = pd.read_csv('data_train/final_locations.csv')
df["dpc"] = list(map(sigmoid, df["dist_pct_ch"]))
df.drop(["hash","vmax","vmin","vmean","time_entry","time_exit","x_home","y_home","nj","dist_pct_ch"],axis=1, inplace=True)
df.set_index("trajectory_id", inplace=True)

y = [df["x_exit"].values, df["y_exit"].values]
y = np.transpose(np.array(y))
df.drop(["x_exit","y_exit"], axis=1, inplace=True)

scaler = MinMaxScaler(feature_range=(-1,1))
X = df.values
X = scaler.fit_transform(X)
print(X[0])
print(y[0])

EPOCHS = 40
BATCH_SIZE = 32 

tf.set_random_seed(0)
np.random.seed(24)

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')

model = None
model = Sequential()

model.add(LSTM(64,activation='relu',return_sequences=False)) #False before Dense layer
model.add(Dropout(0.1))
model.add(BatchNormalization())

# model.add(LSTM(64, activation='relu',kernel_initializer='normal',return_sequences=False))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())

model.add(Dense(1024, activation='relu',kernel_initializer='normal'))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(Dense(2, activation='linear'))

optimiser = tf.keras.optimizers.Adam(lr=1.0, decay=1e-5) #lr=1e-2 = 31.82% valacc


gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
model.compile(loss='mean_absolute_percentage_error', optimizer=optimiser, metrics=['accuracy']) # try different loss functions
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=410,shuffle=True)


model.fit(x_train,y_train, epochs=EPOCHS, validation_split=0.2, callbacks=[checkpoint])

[-0.78302469 -0.42449039 -0.52384475  0.51899512  0.02400801 -0.89947753
 -0.41766156]
[  3744785.44389591 -19281478.34777075]


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'